# Load Libraries

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

from utils_functions import *

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Load Data

In [2]:
#paths
gbye_data_path = '/Users/Serkan/Desktop/personal/jobs/eel_phd/Yellow eel fisher regression project/code/GBYE questionnaire results-anonymised.xlsx'
lnye_data_path = '/Users/Serkan/Desktop/personal/jobs/eel_phd/Yellow eel fisher regression project/code/LNYE Questionnaire results-anonymised.xlsx'

#read data
gbye_raw = pd.read_excel(gbye_data_path)
lnye_raw = pd.read_excel(lnye_data_path)

In [3]:
# keep only relevant columns based on the questionnaire - the common ones
gbye_columns_to_keep = gbye_raw.columns[[1,2,6,7,15,16,17,18, 19, 20,21,22,32, 33, 34, 35, 36, 37,38,44, 45, 46, 47, 49]]
gbye_raw_filtered = gbye_raw[gbye_columns_to_keep]
lnye_columns_to_keep = lnye_raw.columns[[1,2,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,30,31,32,33,37]]
lnye_raw_filtered = lnye_raw[lnye_columns_to_keep]

In [4]:
# rename columns
col_names = ['fishing_years', 'fish_every_year', 'distance_home', 'association', 'hand_net', 'fyke_net', 'fixed_trap', 'wing_net',
             'trawl', 'line', 'spear', 'other', 'mot_money', 'mot_family', 'mot_culture', 'mot_enjoyment', 'mot_taste', 'mot_friendship',
             'mot_professional', 'age', 'sex', 'education', 'other_work', 'income_percentage']
gbye_raw_filtered.columns = col_names
lnye_raw_filtered.columns = col_names

# define feature types
continuous_features = ['distance_home', 'age', 'income_percentage']  
nominal_features = ['fish_every_year', 'association', 'hand_net', 'fyke_net', 'fixed_trap', 'wing_net',
                    'trawl', 'line', 'spear', 'other', 'mot_money', 'mot_family', 'mot_culture', 'mot_enjoyment', 
                    'mot_taste', 'mot_friendship','mot_professional', 'sex', 'other_work']  
ordinal_features = ["fishing_years", 'education']
parameter_types = {
    # Continuous features
    'distance_home': 'continuous',
    'age': 'continuous',
    'income_percentage': 'continuous',
    
    # Nominal features
    'fish_every_year': 'nominal',
    'association': 'nominal',
    'hand_net': 'nominal',
    'fyke_net': 'nominal',
    'fixed_trap': 'nominal',
    'wing_net': 'nominal',
    'trawl': 'nominal',
    'line': 'nominal',
    'spear': 'nominal',
    'other': 'nominal',
    'mot_money': 'nominal',
    'mot_family': 'nominal',
    'mot_culture': 'nominal',
    'mot_enjoyment': 'nominal',
    'mot_taste': 'nominal',
    'mot_friendship': 'nominal',
    'mot_professional': 'nominal',
    'sex': 'nominal',
    'other_work': 'nominal',
    
    # Ordinal features
    'fishing_years': 'ordinal',
    'education': 'ordinal'
}

# Encode parameters

## ordinal mapping

In [5]:
ordinal_mapping = {0: 0, 1: 1, 2: 2, 3:3}
# Map the 'education' column
gbye_raw_filtered.loc[:, 'education'] = gbye_raw_filtered['education'].map(ordinal_mapping)
lnye_raw_filtered.loc[:, 'education'] = lnye_raw_filtered['education'].map(ordinal_mapping)

# Map the 'fishing_years' column
gbye_raw_filtered.loc[:, 'fishing_years'] = gbye_raw_filtered['fishing_years'].map(ordinal_mapping)
lnye_raw_filtered.loc[:, 'fishing_years'] = lnye_raw_filtered['fishing_years'].map(ordinal_mapping)

# Difference between two datasets

## Check the difference

In [6]:
# Compare the two datasets and output the statistics and p-values for each feature
# If p-value is less than 0.05, the feature is considered to be significantly different between the two datasets

comparison_results = compare_datasets(gbye_raw_filtered, lnye_raw_filtered, parameter_types)
print(comparison_results)

            Parameter        Type                Test   Statistic   p-value  \
0       distance_home  continuous  Kolmogorov-Smirnov    0.612554  0.000534   
1                 age  continuous  Kolmogorov-Smirnov    0.235294  0.540139   
2   income_percentage  continuous  Kolmogorov-Smirnov    0.750000  0.000014   
3     fish_every_year     nominal          Chi-Square    0.000000  1.000000   
4         association     nominal          Chi-Square    0.000000  1.000000   
5            hand_net     nominal          Chi-Square    0.000000  1.000000   
6            fyke_net     nominal          Chi-Square    0.000000  1.000000   
7          fixed_trap     nominal          Chi-Square    0.000000  1.000000   
8            wing_net     nominal          Chi-Square    0.000000  1.000000   
9               trawl     nominal          Chi-Square    0.000000  1.000000   
10               line     nominal          Chi-Square    0.000000  1.000000   
11              spear     nominal          Chi-Squar

## Combine datasets 

In [7]:
# merge two dataframes
merged_set = pd.concat([gbye_raw_filtered, lnye_raw_filtered], axis=0)

## Detecting the dataset based on the different variables

### Preprocessing

In [8]:
merged_set_with_class = merged_set.copy()
merged_set_with_class['class'] = [1] * len(gbye_raw_filtered) + [0] * len(lnye_raw_filtered) # 1 gbye 0 lnye
merged_set_with_class = merged_set_with_class[['distance_home', 'income_percentage', 'education', 'class']]

#### handle missing values

In [9]:
# handle the missing values by
drop = False
if drop: # Drop rows with missing values - loosing data which is bad since data size is small
    merged_set_with_class_clean = merged_set_with_class.dropna(subset=['distance_home', 'income_percentage', 'education', 'class'])
    
else: # Fill missing values with the mean for continuous variables and the most frequent value for categorical variables
    # Copy the merged dataset
    merged_set_with_class_clean = merged_set_with_class.copy()
    # Initialize imputer (you can use strategy='mean' or 'median' based on your preference)
    imputer = SimpleImputer(strategy='mean')
    # Apply imputation to the continuous variables
    merged_set_with_class_clean[['distance_home', 'income_percentage']] = imputer.fit_transform(merged_set_with_class_clean[['distance_home', 'income_percentage']])
    # Impute categorical variables with the most frequent value
    imputer = SimpleImputer(strategy='most_frequent')
    merged_set_with_class_clean['education'] = imputer.fit_transform(merged_set_with_class_clean[['education']])

#### ordinal mapping 

In [10]:
ordinal_mapping = {0: 0, 1: 1, 2: 2, 3:3}
merged_set_with_class_clean.loc[:, 'education'] = merged_set_with_class_clean['education'].map(ordinal_mapping)

#### scale the continues variables

In [11]:
scaler = StandardScaler()
# Scale the continuous variables so that they have zero mean and unit variance
merged_set_with_class_clean.loc[:,['distance_home', 'income_percentage']] = scaler.fit_transform(merged_set_with_class_clean[['distance_home', 'income_percentage']])

### Fitting the model

#### Create X and Y

In [12]:
X_dataset_detection = merged_set_with_class_clean[['distance_home', 'income_percentage', 'education']]  # Independent variables
y_dataset_detection = merged_set_with_class_clean['class']  # Dependent variable

#### Fit the model

In [13]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_dataset_detection, y_dataset_detection, test_size=0.2, random_state=42, stratify=y_dataset_detection)
# Add constant to the training and testing sets
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# Fit the logistic regression model using statsmodels on the training data
logreg_sm = sm.Logit(y_train, X_train_const).fit()

# Print the summary to get the p-values for the training data
print(logreg_sm.summary())

Optimization terminated successfully.
         Current function value: 0.178006
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                  class   No. Observations:                   40
Model:                          Logit   Df Residuals:                       36
Method:                           MLE   Df Model:                            3
Date:                Sun, 12 Jan 2025   Pseudo R-squ.:                  0.7177
Time:                        14:36:20   Log-Likelihood:                -7.1202
converged:                       True   LL-Null:                       -25.223
Covariance Type:            nonrobust   LLR p-value:                 6.774e-08
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 5.0728      2.782      1.824      0.068      -0.380      10.525
distanc

#### Accuracy, F1, and other metrics

In [14]:
# Make predictions on the test set
y_pred_prob = logreg_sm.predict(X_test_const)  # Predicted probabilities

# Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
y_pred = (y_pred_prob >= 0.5).astype(int)

# Evaluate the model using scikit-learn metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Accuracy: 0.8181818181818182
F1 Score: 0.6666666666666666
Confusion Matrix:
 [[7 0]
 [2 2]]
Classification Report:
               precision    recall  f1-score   support

           0       0.78      1.00      0.88         7
           1       1.00      0.50      0.67         4

    accuracy                           0.82        11
   macro avg       0.89      0.75      0.77        11
weighted avg       0.86      0.82      0.80        11



# Logistic Regression

## Preprocessing

### Ordinal mapping

In [15]:
ordinal_mapping = {0: 0, 1: 1, 2: 2, 3:3}

merged_set.loc[:, 'education'] = merged_set['education'].map(ordinal_mapping)
merged_set.loc[:, 'fishing_years'] = merged_set['fishing_years'].map(ordinal_mapping)

### Handle missing values

In [16]:
# Handle missing values
drop = False 

if drop: # Drop rows with missing values - loosing data which is bad since data size is small
    merged_set_clean = merged_set.dropna()
    
else: # Fill missing values with the mean for continuous variables and the most frequent value for categorical variables
    # Copy the merged dataset
    merged_set_clean = merged_set.copy()
    # Initialize imputer (you can use strategy='mean' or 'median' based on your preference)
    imputer = SimpleImputer(strategy='mean')
    # Apply imputation to the continuous variables
    merged_set_clean[continuous_features] = imputer.fit_transform(merged_set_clean[continuous_features])
    # Impute categorical variables with the most frequent value
    imputer = SimpleImputer(strategy='most_frequent')
    merged_set_clean[ordinal_features] = imputer.fit_transform(merged_set_clean[ordinal_features])
    merged_set_clean[nominal_features] = imputer.fit_transform(merged_set_clean[nominal_features])

### Scale the continues variables

In [17]:
scaler = StandardScaler()
# Scale the continuous variables
merged_set_clean.loc[:,continuous_features] = scaler.fit_transform(merged_set_clean[continuous_features])

### Encode nominal categorical variables: One-hot encoding

In [18]:
# Convert the nominal features to category
merged_set_clean[nominal_features] = merged_set_clean[nominal_features].astype(int)
merged_set_clean[nominal_features] = merged_set_clean[nominal_features].astype('category')

# Handle column names
x_nominal_features = ['fish_every_year', 'association', 'hand_net', 'fyke_net', 'fixed_trap', 'wing_net',
                      'trawl', 'line', 'spear', 'other', 'sex', 'other_work']
y_column_names = ['mot_culture', 'mot_enjoyment', 'mot_family', 'mot_friendship', 'mot_money', 'mot_professional', 'mot_taste']
y_all = merged_set_clean[y_column_names]

# Drop the target variables from the independent variables
X_unclean = merged_set_clean.drop(y_column_names, axis=1)

# Perform One-Hot Encoding for the nominal categorical columns
X = pd.get_dummies(X_unclean,columns=x_nominal_features, drop_first=True)  # drop_first to avoid multicollinearity

## Feature Selection

In [19]:
X_selection = X.astype(float)

### Correlation Analysis

In [20]:
# Do the correlation analysis to find highly correlated features
# Hihgly correlated features can cause multicollinearity - can affect the model performance in a negative way

# Calculate the correlation matrix
corr_matrix = X_selection.corr()

# Set the threshold for high correlation
threshold = 0.7 # common threshold is 0.7

# Find pairs of features with correlation above the threshold
high_corr_pairs = corr_matrix.unstack().sort_values(ascending=False)

# Set verbose to True to see the full correlation matrix
verbose = False 
if verbose:
    print(f'The correlation matrix is:\n{corr_matrix}\n')

# Filter out the pairs where the correlation is above the threshold and not on the diagonal
high_corr_pairs = high_corr_pairs[(high_corr_pairs > threshold) & (high_corr_pairs < 1)]

# Print the highly correlated pairs
if len(high_corr_pairs) > 0:
    # Drop the highly correlated features
    print(f'The highly correlated pairs are:\n{high_corr_pairs}\n')
else:
    print('There are no highly correlated features in the dataset.')
    

There are no highly correlated features in the dataset.


### Variation Inflation Factor

VIF value should be less than 3

#### First Round

In [21]:
# VIF (Variance Inflation Factor) is used to detect multicollinearity. 
# If VIF is greater than 5, it indicates that the feature is correlated with other features.
# IF VIF is greater than 3, it indicates that the feature is moderately correlated with other features.

# Calculate VIF for each feature
columns_to_drop_vif = []
apply_VIF(X_selection, columns_to_drop_vif)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,feature,VIF
0,const,95.474530
1,fishing_years,1.275230
2,distance_home,2.591698
3,age,1.587947
4,education,1.992052
5,income_percentage,3.963461
6,fish_every_year_1,1.538597
7,association_1,3.660952
8,hand_net_1,1.442330
9,fyke_net_1,5.845047


#### Second Round

In [22]:
# Calculate VIF for each feature
columns_to_drop_vif = ['fyke_net_1']
apply_VIF(X_selection, columns_to_drop_vif)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,feature,VIF
0,const,51.175652
1,fishing_years,1.246837
2,distance_home,2.531825
3,age,1.554293
4,education,1.921012
5,income_percentage,3.861909
6,fish_every_year_1,1.515056
7,association_1,3.097957
8,hand_net_1,1.416410
9,fixed_trap_1,1.287343


#### Third Round

In [23]:
# Calculate VIF for each feature
columns_to_drop_vif = columns_to_drop_vif + ['income_percentage'] # remove VIF > 3
apply_VIF(X_selection, columns_to_drop_vif)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,feature,VIF
0,const,50.235967
1,fishing_years,1.242753
2,distance_home,2.501338
3,age,1.526391
4,education,1.801732
5,fish_every_year_1,1.354492
6,association_1,2.646038
7,hand_net_1,1.229832
8,fixed_trap_1,1.261620
9,wing_net_1,1.952502


There is no variable such that VIF > 3

## Apply Model

### Model 1: Use all variables

Fits the logistic regression model using the training data and evaluates the model using the test data

Fits one logistic regression model for each target variable

In the end, there are 7 logistic regression models

Each regression model uses the same independent variables

The weights of the independent variables are different for each target variable

#### Split data into training and testing sets

In [24]:
# Split data into training and testing sets
Y = y_all.astype(int) # Convert the target variable to integer

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Drop the columns with high VIF and low variability
col_drop = columns_to_drop_vif + ['fixed_trap_1', 'other_1','hand_net_1', 'wing_net_1','line_1']
X_train.drop(columns = col_drop, inplace = True)
X_test.drop(columns = col_drop, inplace = True)

#### Fit the model

In [25]:
# Initialize Logistic Regression model (you can choose a different model here)
logreg = LogisticRegression(max_iter=1000)

# Wrap it with MultiOutputClassifier to handle multiple outputs
multi_output_model = MultiOutputClassifier(logreg, n_jobs=-1)

# Fit the model
multi_output_model.fit(X_train, Y_train)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000), n_jobs=-1)

#### Predictions and Scores

In [26]:
# Make predictions
Y_pred = multi_output_model.predict(X_test)

# Calculate the evaluation metrics
score_metrics = calculate_score_metrics(Y_test, Y_pred, verbose=False)
score_metrics

/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

,Accuracy,F1 Score
mot_culture,0.545455,0.545455
mot_enjoyment,0.727273,0.769231
mot_family,0.818182,0.888889
mot_friendship,0.818182,0.000000
mot_money,0.818182,0.900000
mot_professional,0.727273,0.000000
mot_taste,0.545455,0.000000


#### P-values for the variables

In [27]:
# Calculate the p-values for the features for each target variable
df_significance = calculate_p_value_variable(X_train, Y_train, multi_output_model.estimators_)
df_significance

,intercept,fishing_years,distance_home,age,education,fish_every_year_1,association_1,other_work_1
mot_culture,0.815646,0.358249,0.994653,0.271879,0.230231,0.860676,0.637178,0.59328
mot_enjoyment,0.413461,0.963406,0.844894,0.672241,0.333913,0.724109,0.430651,0.380939
mot_family,0.728462,0.944499,0.347757,0.692139,0.593677,0.650386,0.923663,0.698057
mot_friendship,0.66487,0.844172,0.705626,0.852751,0.259786,0.828,0.788258,0.52699
mot_money,0.485964,0.727825,0.639944,0.083222,0.223093,0.862145,0.44783,0.879009
mot_professional,0.681131,0.933517,0.953672,0.565337,0.610463,0.968565,0.958992,0.971565
mot_taste,0.773793,0.980983,0.75009,0.157383,0.412547,0.679521,0.472467,0.65715


#### P-values for the model

In [28]:
# Calculate the p-values for the models for each target variable
model_p_values = calculate_p_value_model(X_train, Y_train, multi_output_model.estimators_)
model_p_values

/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has fea

,Likelihood Ratio Statistic,p-value
mot_culture,0.1617,1.0000
mot_enjoyment,0.2268,1.0000
mot_family,0.2056,1.0000
mot_friendship,0.1394,1.0000
mot_money,0.5155,0.9994
mot_professional,0.1372,1.0000
mot_taste,0.3066,0.9999


## Model 2: Lasso Regression

Fits lasse regression model - L1 regularization. L1 regularization can be used for feature selection.

L1 regularization can set the coefficients of irrelevant features to zero. This helps to select the most important features.
The regularization strength is controlled by the hyperparameter C. Lower C means stronger regularization.

The model is fitted for each target variable separately.
Therefore, each model has a different set of features.

### Split data into training and test sets

In [29]:
# Split data into training and testing sets
Y = y_all.astype(int) # Convert the target variable to integer

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Drop the columns with high VIF and low variability
col_drop = columns_to_drop_vif + ['fixed_trap_1', 'other_1','hand_net_1', 'wing_net_1','line_1']
X_train.drop(columns = col_drop, inplace = True)
X_test.drop(columns = col_drop, inplace = True)

### Fit the model

In [30]:
# Initialize Logistic Regression with L1 regularization
logreg_l1 = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000, C= 5)

# Fit the model with L1 regularization
multi_output_model = MultiOutputClassifier(logreg_l1, n_jobs=-1)
multi_output_model.fit(X_train, Y_train)

MultiOutputClassifier(estimator=LogisticRegression(C=5, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear'),
                      n_jobs=-1)

### Prediction and Scores

In [31]:
# Make predictions and evaluate
Y_pred = multi_output_model.predict(X_test)

# Calculate the evaluation metrics
score_metrics = calculate_score_metrics(Y_test, Y_pred, verbose = False)
score_metrics

/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

,Accuracy,F1 Score
mot_culture,0.545455,0.545455
mot_enjoyment,0.636364,0.666667
mot_family,0.818182,0.888889
mot_friendship,0.818182,0.000000
mot_money,0.636364,0.777778
mot_professional,0.727273,0.000000
mot_taste,0.545455,0.000000


### P-values for the variables

In [32]:
# Calculate the p-values for the features for each target variable
df_significance = calculate_p_value_variable(X_train, Y_train, multi_output_model.estimators_)
df_significance

,intercept,fishing_years,distance_home,age,education,fish_every_year_1,association_1,other_work_1
mot_culture,1.0,0.318762,0.976648,0.154251,0.079639,0.61452,0.392321,0.543957
mot_enjoyment,0.424308,0.686357,0.949812,0.63025,0.325359,0.662146,0.193585,0.210213
mot_family,0.99402,0.668166,0.196386,0.703264,0.665794,0.390453,0.904765,0.626608
mot_friendship,1.0,NaN,0.609403,0.632948,0.074361,0.893367,0.669591,0.277999
mot_money,0.437019,0.872295,0.35588,0.067761,0.217424,NaN,0.13276,NaN
mot_professional,1.0,NaN,NaN,0.859513,0.489436,NaN,NaN,NaN
mot_taste,1.0,NaN,0.479941,0.105268,0.635421,0.231297,0.153544,0.529866


### P-values for the model

In [33]:
# Calculate the p-values for the models for each target variable
model_p_values = calculate_p_value_model(X_train, Y_train, multi_output_model.estimators_)
model_p_values

/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/Serkan/opt/anaconda3/envs/social_systems/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has fea

,Likelihood Ratio Statistic,p-value
mot_culture,0.1909,1.0000
mot_enjoyment,0.2657,0.9999
mot_family,0.2311,1.0000
mot_friendship,0.1728,1.0000
mot_money,0.5944,0.9990
mot_professional,0.2133,1.0000
mot_taste,0.3766,0.9998
